# Cleanup
The aim of this notebook is to cleanup dataframes that have made in `Extract Data`.

In [1]:
# imports
import pandas as pd
import numpy as np
import os
from importlib import reload
import stage
import stageConstants
from datetime import datetime, timedelta

In [2]:
CREATE_DATA = False
# the first stage of the race
RACE = 'tour-de-france'
YEAR = 2020
STAGE = 1
URL1 = 'https://www.procyclingstats.com/race/{}/{}/stage-{}/result/result'.format(RACE, YEAR, STAGE)

folder_loc = './temp_data/{}'.format(RACE)
if not os.path.exists(folder_loc):
    os.mkdir(folder_loc)
folder_loc = '{}/{}'.format(folder_loc, YEAR)
if not os.path.exists(folder_loc):
    os.mkdir(folder_loc)
    
# saved data locations
NEW_STAGES_LOC = '{}/new_stages_df.csv'.format(folder_loc)
TEAMS_HISTORY_LOC = '{}/teams_history.csv'.format(folder_loc)
RIDERS_LOC = '{}/riders_df.csv'.format(folder_loc)
RACES_LOC = '{}/races_df.csv'.format(folder_loc)
ALL_RACE_DATA = '{}/ALL_RACE_DATA.csv'.format(folder_loc)

CLEANED_RACE_HIST = '{}/cleaned_races_history.csv'.format(folder_loc)
CLEANED_TEAMS = '{}/cleaned_teams.csv'.format(folder_loc)
CLEANED_STAGES = '{}/cleaned_stages.csv'.format(folder_loc)

def reload_libraries():
    reload(stageConstants)
    reload(stage)
    print('Reloaded')

In [3]:
riders_df = pd.read_csv(RIDERS_LOC, index_col='riderID')
races_df = pd.read_csv(RACES_LOC, index_col='raceID')
races_already = pd.read_csv(ALL_RACE_DATA, index_col='Unnamed: 0')

## `riders_df` cleanup
This dataframe has already been cleaned up. 

In [4]:
riders_df.head()

,name,age,url,team
riderID,,,,
0,ALAPHILIPPE JULIAN,28,rider/julian-alaphilippe,DECEUNINCK - QUICK STEP
1,AMADOR ANDREY,34,rider/andrey-amador,INEOS GRENADIERS
2,ANACONA WINNER,32,rider/winner-anacona,TEAM ARKÉA SAMSIC
3,ARNDT NIKIAS,28,rider/nikias-arndt,TEAM SUNWEB
4,ARU FABIO,30,rider/fabio-aru,UAE-TEAM EMIRATES


## `races_df` cleanup
The races that have occured in the season has already been cleaned up. 

In [5]:
races_df.head()

,race,url,cutUrl,year
raceID,,,,
0,Critérium du Dauphiné,race/dauphine/2020/stage-1,race/dauphine/2020,2020
1,Czech Tour,race/czech-cycling-tour/2020/stage-1,race/czech-cycling-tour/2020,2020
2,Etoile de Bessèges,race/etoile-de-besseges/2020/stage-1,race/etoile-de-besseges/2020,2020
3,Jayco Herald Sun Tour,race/herald-sun-tour/2020/stage-1,race/herald-sun-tour/2020,2020
4,La Route d'Occitanie - La Dépêche du Midi,race/la-route-d-occitanie/2020/stage-1,race/la-route-d-occitanie/2020,2020


## `races_already` cleanup
This dataframe must still be cleaned up. 

In [6]:
races_already.reset_index(drop=True, inplace=True)
races_already.rename(columns={'DNF': 'didNotFinish'}, inplace=True)
print(races_already.shape)
races_already.head()

(24922, 30)


,bib,didNotFinish,age,avgTime,countryTeam,createBib,gcPnt,gcPos,gcTime,gcTotalTime,...,teamName,timeAdd,timeWonLost,uciGc,uciStage,url,youthGcTime,youthPos,youthTime,stageType
0,205,NaN,29.0,NaN,NaN,False,NaN,NaN,NaN,NaN,...,Team Sunweb,NaN,NaN,NaN,400.0,rider/michael-matthews,NaN,NaN,NaN,0
1,122,NaN,32.0,NaN,NaN,False,NaN,NaN,NaN,NaN,...,Mitchelton-Scott,NaN,NaN,NaN,320.0,rider/luka-mezgec,NaN,NaN,NaN,0
2,114,NaN,27.0,NaN,NaN,False,NaN,NaN,NaN,NaN,...,Deceuninck - Quick Step,NaN,NaN,NaN,260.0,rider/florian-senechal,NaN,NaN,NaN,0
3,222,NaN,26.0,NaN,NaN,False,NaN,NaN,NaN,NaN,...,Circus - Wanty Gobert,NaN,NaN,NaN,220.0,rider/aime-de-gendt,NaN,NaN,NaN,0
4,231,NaN,24.0,NaN,NaN,False,NaN,NaN,NaN,NaN,...,NIPPO DELKO One Provence,NaN,NaN,NaN,180.0,rider/alessandro-fedeli,NaN,NaN,NaN,0


From the cell above we see that the `races_already` data frame has 24 922 entries. Many of the collumns are empty and contain NaN values. These collumns must be improved on or deleted from the data frame.

In [7]:
# 3 rows are empty
races_already = races_already[~races_already.name.isnull()]
# createBib not always created as column
races_already.createBib.fillna(False, inplace=True)

# filling points nan as 0
races_already.uciGc.fillna(0, inplace=True)
races_already.greenPnts.fillna(0, inplace=True)
races_already.pnt.fillna(0, inplace=True)
races_already.komPnts.fillna(0, inplace=True)
races_already.uciStage.fillna(0, inplace=True)

# fill nan didNotFinish classification
races_already.didNotFinish.fillna('', inplace=True)
races_already['didNotFinish'] = races_already.didNotFinish.str.replace('\xa0\xa0', '')

In [8]:
# fill empty team name values from country name values from other columns
# in some races a rider competes under his country. In other races he competes from his team
# Sometimes riders are unsigned and do not compete under a team name. 
try:
    races_already['teamName'] = np.where(races_already['teamName'].isnull(), races_already['countryTeam'], races_already['teamName'])
    races_already.loc[races_already.countryTeam.isnull(), 'country'] = 0
    races_already.loc[~races_already.countryTeam.isnull(), 'country'] = 1
except KeyError:
    print('"countryTeam" already removed.')
races_already['teamName'] = races_already.teamName.fillna('')

# fill empty position values with values from stage positions
races_already['gcPos'] = np.where(races_already['gcPos'].isnull(), races_already['stagePos'], races_already['gcPos'])
races_already['stagePos'] = np.where(races_already['stagePos'].isnull(), races_already['gcPos'], races_already['stagePos'])

# fill k.o.m and green points positions with maximum positions. create 'kom', 'green', 'youth' binary columns
# k.o.m
races_already.loc[races_already.greenPos.isnull(), 'kom'] = 0
races_already.loc[~races_already.greenPos.isnull(), 'kom'] = 1
maxKOMPos = races_already.komPos.max() + 1
races_already['komPos'] = np.where(races_already['komPos'].isnull(), maxKOMPos, races_already['komPos'])

# green
races_already.loc[races_already.greenPos.isnull(), 'green'] = 0
races_already.loc[~races_already.greenPos.isnull(), 'green'] = 1
maxGreenPos = races_already.greenPos.max() + 1
races_already['greenPos'] = np.where(races_already['greenPos'].isnull(), maxGreenPos, races_already['greenPos'])

# youth
races_already.loc[races_already.youthPos.isnull(), 'youth'] = 0
races_already.loc[~races_already.youthPos.isnull(), 'youth'] = 1
maxYouthPos = races_already.youthPos.max() + 1
races_already['youthPos'] = np.where(races_already['youthPos'].isnull(), maxYouthPos, races_already['youthPos'])

In [9]:
# time columns currently non-numeric, '+' and '-' symbols that must be removed. 
races_already['gcTotalTime'] = races_already.gcTotalTime.str.replace('+', '')
races_already['gcTotalTime'] = races_already.gcTotalTime.str.replace('-', '')
races_already['gcTime'] = races_already.gcTime.str.replace('-', '')
races_already['gcTime'] = races_already.gcTime.str.replace('+', '')
races_already['stageTime'] = races_already.stageTime.str.replace('-', '')
races_already['stageTime'] = races_already.stageTime.str.replace('+', '')

races_already['stageTime'] = np.where(races_already['stageTime'] == '-', races_already['gcTotalTime'], races_already['stageTime'])
races_already['stageTime'] = np.where(races_already['stageTime'].isnull(), races_already['gcTime'], races_already['stageTime'])

# fill NaN values in collumns from other time collumns based on stage type specifications
gcstg = races_already[((races_already.stageType == stageConstants.ONE_DAY_RACE) | \
                     (races_already.stageType == stageConstants.FIRST_STAGE_IN_TOUR) | \
                     (races_already.stageType == stageConstants.PROLOGUE) | \
                     (races_already.stageType == stageConstants.ITT_CC)) & (races_already.gcTime.isnull())][['stageTime']]
races_already.loc[gcstg.index, 'gcTime'] = gcstg.stageTime

gct = races_already[((races_already.stageType == stageConstants.OTHER_TOUR_STAGE) | \
                     (races_already.stageType == stageConstants.ITT)) & (races_already.gcTime.isnull())][['gcTotalTime']]
races_already.loc[gct.index, 'gcTime'] = gct.gcTotalTime

gctt = races_already[((races_already.stageType == stageConstants.ONE_DAY_RACE) | \
                      (races_already.stageType == stageConstants.TTT) | \
                      (races_already.stageType == stageConstants.ITT_CC)) & (races_already.gcTotalTime.isnull())][['gcTime']]
races_already.loc[gctt.index, 'gcTotalTime'] = gctt['gcTime']

In [10]:
# remove empty/ unnecessary columns
for column in ['timeWonLost', 'avgTime', 'countryTeam', 'gcPnt', 'stgAvgPace', 'timeAdd', 'youthGcTime', 'youthTime']:
    try:
        races_already.drop(column, axis=1, inplace=True)
    except:
        print('"{}" already removed'.format(column))

# fix columns types
races_already = races_already.astype({'uciGc': int, \
                                      #'age': int, \
                                        'stageID': int, 'greenPnts': int, 'stagePos': int, 'gcPos': int, 'greenPos': int, \
                                        'komPnts': int, 'komPos': int, 'pnt': int, 'stagePos': int, 'uciStage': int, \
                                        'country': bool, 'youth': bool, 'kom': bool, 'youthPos': bool, 'green': bool})
print(races_already.shape)
races_already.head()

(24919, 26)


,bib,didNotFinish,age,createBib,gcPos,gcTime,gcTotalTime,greenPnts,greenPos,komPnts,...,teamName,uciGc,uciStage,url,youthPos,stageType,country,kom,green,youth
0,205,,29.0,False,1,6:01:14,6:01:14,0,117,0,...,Team Sunweb,0,400,rider/michael-matthews,True,0,False,False,False,False
1,122,,32.0,False,2,0:01,0:01,0,117,0,...,Mitchelton-Scott,0,320,rider/luka-mezgec,True,0,False,False,False,False
2,114,,27.0,False,3,0:01,0:01,0,117,0,...,Deceuninck - Quick Step,0,260,rider/florian-senechal,True,0,False,False,False,False
3,222,,26.0,False,4,0:01,0:01,0,117,0,...,Circus - Wanty Gobert,0,220,rider/aime-de-gendt,True,0,False,False,False,False
4,231,,24.0,False,5,0:01,0:01,0,117,0,...,NIPPO DELKO One Provence,0,180,rider/alessandro-fedeli,True,0,False,False,False,False


In [11]:
def fix_time_column(df, time_col):
    df[time_col] = df[time_col].str.replace(' ', '')
    df.loc[df[time_col] == ',,', time_col] = None
    df[time_col] = df[time_col].fillna(method='ffill')

    l4 = df[df[time_col].str.len() == 4]
    df.loc[l4.index, time_col] = '00:0' + l4[time_col].astype(str)

    l5 = df[df[time_col].str.len() == 5]
    df.loc[l5.index, time_col] = '00:' + l5[time_col].astype(str)
    return df
    
def create_final_time_col(time_df, time_col, pos_col):
    #'gcTotalTime'
    ''' Clean up the time column. Determine the days, hours and true time of a race.
    @retuirn a dataframe with the final "time" column determine'''
    first_time = time_df[time_df[pos_col] == 1]
    first_time.loc[first_time.index, 'gtt'] = first_time[time_col]

    time_df.loc[first_time.index, 'gtt'] = first_time['gtt']
    time_df['gtt'] = time_df['gtt'].fillna(method='ffill')

    time_df['gix'] = time_df.gtt.str.index(':')
    time_df['allHours'] = time_df.apply(lambda x: int(int(x['gtt'][:x['gix']])), axis=1)
    time_df['days'] = (time_df['allHours'].astype(int) / 24).astype(int)

    time_df['hours'] = time_df['allHours'].astype(int) % 24
    time_df['gtt1'] = time_df.apply(lambda x: str(x['hours']) + x['gtt'][x['gix']:], axis=1)

    add_df = time_df[time_df[time_col] != time_df.gtt]
    time_df.loc[add_df.index, 'sum'] = add_df[time_col]
    time_df['sum'] = time_df['sum'].fillna('0:00:00')
    time_df['sum'] = time_df['sum'].str.replace(' ', '')
    
    d = pd.to_timedelta(time_df['days'].astype(str) + 'day')
    g = pd.to_timedelta(time_df['gtt1'])
    s = pd.to_timedelta(time_df['sum'])

    time_df.loc[:, 'time'] = pd.DataFrame(d + g + s, columns=['time'])
    return time_df

# fix/ clean 'gcTotalTime' column and create the final 'time' column to how much time 
# the rider has spent on the race
races_already = fix_time_column(races_already, 'gcTotalTime')
time_df = create_final_time_col(races_already[['bib', 'name', 'gcTotalTime', 'stagePos', 'gcTime', 'stageID', \
                                               'stageTime', 'stageType', 'didNotFinish']].copy(), 'gcTotalTime', 'stagePos')
races_already.loc[time_df.index, 'time'] = time_df['time'].copy()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [12]:
try:
    # create binary collumns for each 'DNF' classifcation
    races_already.didNotFinish.unique()
    for dnfReason in races_already.didNotFinish.unique():
        if dnfReason != '':
            races_already.loc[races_already.didNotFinish == dnfReason, dnfReason] = 1
            races_already.loc[races_already.didNotFinish != dnfReason, dnfReason] = 0
            races_already[dnfReason] = races_already[dnfReason].astype('bool')
    races_already.drop(['didNotFinish', 'gcTime'], axis=1, inplace=True)
except AttributeError:
    print('"didNotFinish" already removed')

In [13]:
races_already.iloc[:, :15].head(10)

,bib,age,createBib,gcPos,gcTotalTime,greenPnts,greenPos,komPnts,komPos,name,pnt,stageID,stagePos,stageTime,teamName
0,205,29.0,False,1,6:01:14,0,117,0,46,Matthews Michael,225,0,1,6:01:14,Team Sunweb
1,122,32.0,False,2,00:00:01,0,117,0,46,Mezgec Luka,150,0,2,0:01,Mitchelton-Scott
2,114,27.0,False,3,00:00:01,0,117,0,46,Sénéchal Florian,110,0,3,0:01,Deceuninck - Quick Step
3,222,26.0,False,4,00:00:01,0,117,0,46,De Gendt Aimé,90,0,4,0:01,Circus - Wanty Gobert
4,231,24.0,False,5,00:00:01,0,117,0,46,Fedeli Alessandro,80,0,5,0:01,NIPPO DELKO One Provence
5,217,19.0,False,6,00:00:01,0,117,0,46,Simmons Quinn,70,0,6,0:01,Trek - Segafredo
6,202,22.0,False,7,00:00:03,0,117,0,46,Eekhoff Nils,60,0,7,0:03,Team Sunweb
7,187,28.0,False,8,00:00:05,0,117,0,46,McLay Daniel,50,0,8,0:05,Team Arkéa Samsic
8,175,33.0,False,9,00:00:05,0,117,0,46,Roux Anthony,46,0,9,0:05,Groupama - FDJ
9,65,24.0,False,10,00:00:05,0,117,0,46,García Cortina Iván,42,0,10,0:05,Bahrain - McLaren


In [14]:
races_already.iloc[:, 16:].head(10)

,uciStage,url,youthPos,stageType,country,kom,green,youth,time,DNF,DNS,OTL,DSQ
0,400,rider/michael-matthews,True,0,False,False,False,False,06:01:14,False,False,False,False
1,320,rider/luka-mezgec,True,0,False,False,False,False,06:01:15,False,False,False,False
2,260,rider/florian-senechal,True,0,False,False,False,False,06:01:15,False,False,False,False
3,220,rider/aime-de-gendt,True,0,False,False,False,False,06:01:15,False,False,False,False
4,180,rider/alessandro-fedeli,True,0,False,False,False,False,06:01:15,False,False,False,False
5,140,rider/quinn-simmons,True,0,False,False,False,False,06:01:15,False,False,False,False
6,120,rider/nils-eekhoff,True,0,False,False,False,False,06:01:17,False,False,False,False
7,100,rider/daniel-mclay,True,0,False,False,False,False,06:01:19,False,False,False,False
8,80,rider/anthony-roux,True,0,False,False,False,False,06:01:19,False,False,False,False
9,68,rider/ivan-garcia-cortina,True,0,False,False,False,False,06:01:19,False,False,False,False


The collumns in `races_already` have been cleaned. The only collumns with NaN values is the `age` collumn. We will examine in the next notebook whether this problem persists. 

In [15]:
# columns that still has an error
races_already[races_already.age.isnull()][['name', 'age']].head()

,name,age
1465,Barratt Elliot,NaN
2712,Costa Fabian,NaN
2734,Schwarzl Philipp,NaN
2739,Zangerle Emanuel,NaN
2747,Heigl Philipp,NaN


In [16]:
name = races_already.loc[23588, 'name']
races_already[races_already.name == name][['name', 'bib', 'age']]

,name,bib,age
8126,Obando Renzo,206,NaN
11619,Obando Renzo,206,NaN
13162,Obando Renzo,206,NaN
19025,Obando Renzo,206,NaN
21955,Obando Renzo,206,NaN
22233,Obando Renzo,206,NaN
23588,Obando Renzo,206,NaN


In [17]:
races_already.to_csv(CLEANED_RACE_HIST)

## `teams_hist_df` cleanup
The teams' past races' success has already been cleaned. 

In [18]:
teams_hist_df = pd.read_csv(TEAMS_HISTORY_LOC, index_col='Unnamed: 0')
teams_hist_df.head()

,teamPos,teamName,teamTime,stageID
0,1,Team Arkéa Samsic,52:07:18,141
1,2,Groupama - FDJ,1:01,141
2,3,Team Sunweb,3:52,141
3,4,Caja Rural - Seguros RGA,5:00,141
4,5,"Cofidis, Solutions Crédits",7:41,141


In [19]:
th_df = fix_time_column(teams_hist_df, 'teamTime')
teams_hist_df = create_final_time_col(th_df, 'teamTime', 'teamPos')
teams_hist_df.drop(['gtt', 'gix', 'allHours', 'days', 'hours', 'gtt1', 'sum'], axis=1, inplace=True)
teams_hist_df.head()

,teamPos,teamName,teamTime,stageID,time
0,1,Team Arkéa Samsic,52:07:18,141,2 days 04:07:18
1,2,Groupama - FDJ,00:01:01,141,2 days 04:08:19
2,3,Team Sunweb,00:03:52,141,2 days 04:11:10
3,4,Caja Rural - Seguros RGA,00:05:00,141,2 days 04:12:18
4,5,"Cofidis, Solutions Crédits",00:07:41,141,2 days 04:14:59


In [20]:
teams_hist_df.to_csv(CLEANED_TEAMS)

## `stages_df` cleanup

In [62]:
stages_df = pd.read_csv(NEW_STAGES_LOC, index_col='Unnamed: 0')
print('{} == {}'.format(stages_df.shape[0], len(stages_df.index.unique())))
stages_df.head()

193 == 193


,stage,url,date,distance,cutUrl,raceID,stageType,dateFull,averageSpeed,raceCtgr,parcoursType,PCSPointScale,start,end,mountains,numMount,raceRank,racePoints
0,Bretagne Classic - Ouest-France,race/bretagne-classic/2020/result,25.08,247.75,race/bretagne-classic/2020,27,0,25th August 2020,41.15 km/h,Men Elite,45,1.WT.B,Plouay,Plouay,"['Restergal', 'Côte du Lezot', 'Restergal', 'C...",5.0,35.0,390 pnt
1,Cadel Evans Great Ocean Road Race,race/great-ocean-race/2020/result,2.02,171.10,race/great-ocean-race/2020,28,0,2nd February 2020,41.76 km/h,Men Elite,53,1.WT.C,Geelong,Geelong,[],0.0,55.0,395 pnt
2,Circuito de Getxo-Memorial Hermanos Otxoa,race/circuito-de-getxo/2020/result,2.08,177.00,race/circuito-de-getxo/2020,29,0,2nd August 2020,42.29 km/h,Men Elite,0*,1.1,Getxo,Getxo,[],0.0,64.0,238 pnt
3,Clasica de Almeria,race/clasica-de-almeria/2020/result,16.02,187.60,race/clasica-de-almeria/2020,30,0,16th February 2020,42.63 km/h,Men Elite,0*,1.HC,Roquetas de Mar,Roquetas de Mar,[],0.0,56.0,397 pnt
4,European Continental Championships - ITT,race/uec-road-european-championships-itt/2020/...,24.08,25.60,race/uec-road-european-championships-itt/2020,31,6,24th August 2020,50.69 km/h,Men Elite,0*,UCI.Cont.Ch.TT,Plouay,Plouay,[],0.0,104.0,86 pnt


In [63]:
t = races_already.groupby(['stageID', 'stageType'])[['name']].count()
for _id, _type in t.index:
    stages_df.loc[_id, 'stageType'] = _type

In [64]:
stages_df['racePoints'] = stages_df.racePoints.str.replace('pnt', '')
#'raceRank' is relative to when the rank is taken
# not constant or reliable
stages_df.index.name = 'stageID'

stages_df.numMount.fillna(0, inplace=True)
stages_df.mountains.fillna('', inplace=True)
stages_df.racePoints.fillna(0, inplace=True)

# average speed
stages_df['averageSpeed'] = stages_df.averageSpeed.str.replace(' km/h', '')

stages_df = stages_df.astype({'numMount': int, 'averageSpeed': float})
stages_df.drop(['raceCtgr', 'date', 'raceRank', 'cutUrl'], axis=1, inplace=True)

stages_df.head()

,stage,url,distance,raceID,stageType,dateFull,averageSpeed,parcoursType,PCSPointScale,start,end,mountains,numMount,racePoints
stageID,,,,,,,,,,,,,,
0,Bretagne Classic - Ouest-France,race/bretagne-classic/2020/result,247.75,27,0,25th August 2020,41.15,45,1.WT.B,Plouay,Plouay,"['Restergal', 'Côte du Lezot', 'Restergal', 'C...",5,390
1,Cadel Evans Great Ocean Road Race,race/great-ocean-race/2020/result,171.10,28,0,2nd February 2020,41.76,53,1.WT.C,Geelong,Geelong,[],0,395
2,Circuito de Getxo-Memorial Hermanos Otxoa,race/circuito-de-getxo/2020/result,177.00,29,0,2nd August 2020,42.29,0*,1.1,Getxo,Getxo,[],0,238
3,Clasica de Almeria,race/clasica-de-almeria/2020/result,187.60,30,0,16th February 2020,42.63,0*,1.HC,Roquetas de Mar,Roquetas de Mar,[],0,397
4,European Continental Championships - ITT,race/uec-road-european-championships-itt/2020/...,25.60,31,6,24th August 2020,50.69,0*,UCI.Cont.Ch.TT,Plouay,Plouay,[],0,86


### `stages_df` errors
From the code above we see that there are stages that are erroneously made:
 - Stages that have location names as PCSPointScale values (id `75`)

In [65]:
# errors
error_df = stages_df[(stages_df.PCSPointScale.str.match(r'[\w\d\.]*\.{1}[\w\d\.]+')==False) & \
                     (stages_df.PCSPointScale != 'E-CC')]
error_df

,stage,url,distance,raceID,stageType,dateFull,averageSpeed,parcoursType,PCSPointScale,start,end,mountains,numMount,racePoints
stageID,,,,,,,,,,,,,,
75,Stage 10 - île d'Oléron (Le Château-d'Oléron) ...,race/tour-de-france/2020/stage-10,168.5,13,2,8th September 2020,46.94,1,île d'Oléron (Le Château-d'Oléron),Île de Ré (Saint-Martin-de-Ré),2,['1508 pnt'],1,0
185,Stage 8 - Cazères › Loudenvielle,race/tour-de-france/2020/stage-8,141.0,13,2,5th September 2020,34.93,263,Cazères,Loudenvielle,Col de Peyresourde,"['Port de Balès', 'Col de Menté']",2,1508
187,Stage 9 - Pau › Laruns,race/tour-de-france/2020/stage-9,153.0,13,2,6th September 2020,39.02,212,Pau,Laruns,Col de Marie-Blanque,"[""Col d'Ichère"", 'Col de Soudet', 'Col de la H...",4,1508


In [66]:
for eix, err_row in error_df.iterrows():
    if err_row['PCSPointScale'] == 'E-CC':
        continue
    start = err_row['PCSPointScale']
    end = err_row['start']
    mount = err_row['end']
    mountains = err_row['mountains']
    
    stages_df.loc[eix, 'PCSPointScale'] = ''
    stages_df.loc[eix, 'start'] = start
    stages_df.loc[eix, 'end'] = end
    
    if stage.is_not_int(mount):
        mountains = '["{}", {}'.format(mount, mountains[1:])
        stages_df.loc[eix, 'mountains'] = mountains
        stages_df.loc[eix, 'numMount'] = len(mountains.split(','))
    elif 'pnt' in mountains[2:-2]:
        stages_df.loc[eix, 'mountains'] = '[]'
        stages_df.loc[eix, 'numMount'] = 0
        stages_df.loc[eix, 'racePoints'] = mountains[2:-2].replace(' pnt', '')
    else:
        stages_df.loc[eix, 'mountains'] = '[]'
        stages_df.loc[eix, 'numMount'] = 0
        stages_df.loc[eix, 'racePoints'] = mount

stages_df[stages_df.PCSPointScale.str.match(r'[\w\d\.]*\.{1}[\w\d\.]+')==False]

,stage,url,distance,raceID,stageType,dateFull,averageSpeed,parcoursType,PCSPointScale,start,end,mountains,numMount,racePoints
stageID,,,,,,,,,,,,,,
5,European Continental Championships - Road Race,race/uec-road-european-championships/2020/result,177.45,32,0,26th August 2020,42.19,45,E-CC,Plouay,Plouay,[],0,569
75,Stage 10 - île d'Oléron (Le Château-d'Oléron) ...,race/tour-de-france/2020/stage-10,168.50,13,2,8th September 2020,46.94,1,,île d'Oléron (Le Château-d'Oléron),Île de Ré (Saint-Martin-de-Ré),[],0,1508
185,Stage 8 - Cazères › Loudenvielle,race/tour-de-france/2020/stage-8,141.00,13,2,5th September 2020,34.93,263,,Cazères,Loudenvielle,"[""Col de Peyresourde"", 'Port de Balès', 'Col d...",3,1508
187,Stage 9 - Pau › Laruns,race/tour-de-france/2020/stage-9,153.00,13,2,6th September 2020,39.02,212,,Pau,Laruns,"[""Col de Marie-Blanque"", ""Col d'Ichère"", 'Col ...",5,1508


On further reading, it is gound that `ParcoursType` is a profile score for the stage. More can be read about the calcuation and meaning at https://www.procyclingstats.com/race/tour-de-france/2020/stage-8/today/profile-score . Stages that have been given a `0*` rating have not been scored.

In [67]:
stages_df.parcoursType.unique()

array(['45', '53', '0*', '158', '225', '101', '77', '96', '229', '16',
       '48', '370', '116', '28', '152', '24', '102', '126', '25', '3',
       '107', '22', '36', '34', '31', '18', '203', '4', '12', '1', '6',
       '85', '32', '15', '14', '78', '182', '153', '5', '193', '66',
       '283', '73', '140', '281', '265', '33', '299', '242', '386', '216',
       '37', '100', '95', '10', '239', '65', '144', '269', '7', '67',
       '136', '61', '305', '173', '257', '185', '240', '139', '183', '43',
       '46', '109', '209', '86', '127', '26', '162', '227', '263', '212',
       '104', '17', '113'], dtype=object)

In [68]:
# should have no entries
stages_df[stages_df.parcoursType.isnull()]

,stage,url,distance,raceID,stageType,dateFull,averageSpeed,parcoursType,PCSPointScale,start,end,mountains,numMount,racePoints
stageID,,,,,,,,,,,,,,


In [69]:
stages_df.loc[stages_df.parcoursType != '0*', 'scored'] = 1
stages_df.loc[stages_df.parcoursType == '0*', 'scored'] = 0
stages_df['parcoursType'] = stages_df.parcoursType.str.replace('0\*', '0')
stages_df = stages_df.astype({'parcoursType': int, 'scored':  bool})
stages_df.parcoursType.unique()

stage             object
url               object
distance         float64
raceID             int64
stageType          int64
dateFull          object
averageSpeed     float64
parcoursType       int64
PCSPointScale     object
start             object
end               object
mountains         object
numMount           int64
racePoints        object
scored              bool
dtype: object

In [70]:
stages_df.PCSPointScale.unique()

array(['1.WT.B', '1.WT.C', '1.1', '1.HC', 'UCI.Cont.Ch.TT', 'E-CC', '1.2',
       '1.WT.A', 'Nationals.C3', 'Nationals.C3.TT', '2.1.Stage',
       '2.1.TTT', '2.HC.Stage', '2.2.Stage', '2.WT.Stage', 'GT.A.Stage',
       '2.WT.C.Stage', ''], dtype=object)

`PCSPointScale` is classification of the stage type according to UCI rankings. 

`PCSPointScale` with the word *Stage* in label should be removed. The classification does not include this substring. 

There are 3 stages that have no ranking. Further investigation on these 3 stages indicates that they are **Tour de France** stages. Hence they should get the same classification as the other Tour de France stages. 


In [71]:
stages_df['PCSPointScale'] = stages_df.PCSPointScale.str.replace('.Stage', '')
stages_df.loc[stages_df.PCSPointScale == '', 'PCSPointScale'] = 'GT.A'
stages_df.head()

,stage,url,distance,raceID,stageType,dateFull,averageSpeed,parcoursType,PCSPointScale,start,end,mountains,numMount,racePoints,scored
stageID,,,,,,,,,,,,,,,
0,Bretagne Classic - Ouest-France,race/bretagne-classic/2020/result,247.75,27,0,25th August 2020,41.15,45,1.WT.B,Plouay,Plouay,"['Restergal', 'Côte du Lezot', 'Restergal', 'C...",5,390,True
1,Cadel Evans Great Ocean Road Race,race/great-ocean-race/2020/result,171.10,28,0,2nd February 2020,41.76,53,1.WT.C,Geelong,Geelong,[],0,395,True
2,Circuito de Getxo-Memorial Hermanos Otxoa,race/circuito-de-getxo/2020/result,177.00,29,0,2nd August 2020,42.29,0,1.1,Getxo,Getxo,[],0,238,False
3,Clasica de Almeria,race/clasica-de-almeria/2020/result,187.60,30,0,16th February 2020,42.63,0,1.HC,Roquetas de Mar,Roquetas de Mar,[],0,397,False
4,European Continental Championships - ITT,race/uec-road-european-championships-itt/2020/...,25.60,31,6,24th August 2020,50.69,0,UCI.Cont.Ch.TT,Plouay,Plouay,[],0,86,False


In [72]:
stages_df.to_csv(CLEANED_STAGES)

# Conclusion
`riders_df`, `races_df`, `races_already`, `teams_hist_df`, `stages_df` have been cleaned.